In [2]:
import pandas as pd
from dotenv import load_dotenv

from pydantic import BaseModel
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv(
    dotenv_path="/Users/tomwattley/App/racing-api-project/racing-api-project/libraries/api-helpers/src/api_helpers/.env"
)


True

In [3]:
import numpy as np

def bootstrap_stat(x, n_boot=10000, stat="mean", seed=42, ci=(2.5, 97.5)):
    x = np.asarray(x, dtype=float)
    x = x[~np.isnan(x)]  # drop NaNs
    rng = np.random.default_rng(seed)
    n = x.size
    idx = rng.integers(0, n, size=(n_boot, n))  # bootstrap indices

    samples = x[idx]
    if stat == "mean":
        stats = samples.mean(axis=1)
    elif stat == "median":
        stats = np.median(samples, axis=1)
    else:
        # custom callable: stat(samples, axis=1)
        stats = stat(samples, axis=1)

    ci_vals = np.percentile(stats, ci) if ci else None
    return stats, ci_vals

# Example
data = [80.0, 89.0, 80.0, 63.0, 81.0, 64.0, np.nan, 60.0, 83.0, 72.0]
stats, ci = bootstrap_stat(data, n_boot=100000, stat="mean")
print(f"Bootstrap mean CI 95%: {ci}")

Bootstrap mean CI 95%: [68.22222222 80.88888889]
